# 决策树之CART算法

scikit-learn使用优化版的CART算法作为其决策树算法的实现

## 基尼系数

ID3和C4.5算法中熵计算涉及大量对数运算，使用基尼系数替代信息增益比作为选择特征的标准，将大大减小计算量。**基尼系数代表模型的不纯度，基尼系数越小，不纯度越低，特征越好**。

基尼系数表达式：
$$ Gini(p) = \sum_{k=1}^K p_k(1-p_k) = 1 - \sum_{k=1}^K p_k^2 $$

给定样本集$D$，共有$K$个类别，第$k$个类别的数量为$C_k$，则样本$D$的基尼系数为：
$$ Gini(D) = 1 - \sum_{k=1}^K{(\frac{\mid C_k \mid}{\mid D \mid})}^2 $$

如果根据特征$A$对样本集合$D$进行划分，分成$D_1$和$D_2$两部分，则在特征$A$的条件下，$D$的基尼系数为：
$$ Gini(D, A) = \frac{\mid D_1 \mid}{\mid D \mid} Gini(D_1) + \frac{\mid D_2 \mid}{\mid D \mid} Gini(D_2) $$

实际上，基尼系数非常接近熵之半，可采用基尼系数近似。

## CART分类树对特征处理

1. 连续特征处理。思路同C4.5算法相似

2. 离散特征处理。二分类处理离散特征，如特征A有三种取值a、b、c，二分类有三种情况：{a}和{b、c}、{a、b}和{c}、{a、c}和{b}，分别计算三种情况的基尼系数，选择最小的组合，建立二叉树。


CART分类树是二叉树，ID3和C4.5算法是多叉树

## CART分类树算法流程

输入：训练集$D$，基尼系数阈值，样本个数阈值

输出：决策树$T$

算法从根节点开始，用训练集递归建立CART树

1. 对于当前节点的数据集为$D$，如果样本个数小于阈值或者没有特征，则返回决策子树，当前节点停止递归。

2. 计算样本集$D$的基尼系数，如果基尼系数小于阈值，则返回决策树子树，当前节点停止递归。

3. 计算当前节点现有的各个特征的各个特征值对数据集$D$的基尼系数，对于离散值和连续值的处理方法和基尼系数的计算见上节。

4. 在计算出来的各个特征的各个特征值对数据集$D$的基尼系数中，选择基尼系数最小的特征$A$和对应的特征值a。根据这个最优特征和最优特征值，把数据集划分成两部分$D_1$和$D_2$，同时建立当前节点的左右节点，做节点的数据集$D$为$D_1$，右节点的数据集$D$为$D_2$。

5. 对左右的子节点递归的调用1-4步，生成决策树。


**对于生成的决策树做预测的时候，假如测试集里的样本A落到了某个叶子节点，而节点里有多个训练样本。则对于A的类别预测采用的是这个叶子节点里概率最大的类别**

## CART回归树算法流程

CART回归树算法和分类树算法类似。若样本输出是离散值，则为分类树；若为连续值，则为回归树。

CART回归树与分类树不同点为：

1. 特征处理方式不同。连续特征需要统计划分点，利用划分点对特征值进行二分类。

2. 特征节点选择方式不同。回归树利用均方差和作为分类依据（计算被节点划分后两个数据子集各自均方差，选择均方差和最小的节点分类），分类树则利用基尼系数作为分类依据。

## CART树剪枝

> CART算法很容易产生过拟合，泛化能力变差，必须对CART树进行剪枝，以此来增加决策树的泛化能力。CART采用的是后剪枝法。

剪枝过程中，任意一刻子树T损失函数为：
$$ C_\alpha (T_t) = C(T_t) + \alpha \mid T_t \mid $$
其中，$\alpha$为正则化参数，$C(T_t)$为训练数据的预测误差，$\mid T_t \mid$为子树T的叶子节点的数量

> 对于固定的$\alpha$一定存在是损失函数$ C_\alpha (T_t) $最小的唯一子树

需要先计算节点t的子树是否被剪枝条件下损失函数大小，若未剪枝损失函数值大于剪枝损失函数值，则进行剪枝。如下：
$$ C(T_t) + \alpha \mid T_t \mid = C_\alpha (T_t) >= C_\alpha (T) = C(T) + \alpha $$, 即 $$ \frac{C(T)-C(T_t)}{\mid T_t \mid - 1} =< \alpha $$


输入：CART算法建立的原始决策树T

输出：最优决策子树$T_\alpha$

算法过程如下：

1. 初始化$α_min = \infty $， 最优子树集合$\omega = \lbrace T \rbrace$

2. 从叶子节点开始自下而上计算各内部节点t的训练误差损失函数$C_\alpha (T_t)$（回归树为均方差，分类树为基尼系数）, 叶子节点数$\mid T_t \mid$，以及正则化阈值$ \alpha=min \lbrace \frac{C(T) - C(T_t)}{|T_t| - 1}, \alpha_min \rbrace $, 更新$ \alpha_min = \alpha $

3. 得到所有节点的$\alpha$值的集合M。

4. 从M中选择最大的值$\alpha_k$，自上而下的访问子树t的内部节点，如果$ \frac{C(T) - C(T_t)}{|T_t| - 1} \leq \alpha_k $时，进行剪枝。并决定叶节点t的值。如果是分类树，则是概率最高的类别，如果是回归树，则是所有样本输出的均值。这样得到$\alpha_k$对应的最优子树$T_k$

5. 最优子树集合$\omega = \omega \cup T_k$，$M = M - \lbrace \alpha_k \rbrace$。

6. 如果M不为空，则回到步骤4。否则就已经得到了所有的可选最优子树集合$\omega$.

7. 采用交叉验证在$\omega$选择最优子树$T_\alpha$


**CART算法缺点：**

1. 在选择特征时，只考虑选择一个最优特征，而不是选择最优的一个特征线性组合（多变量决策，multi-variate decision tree）

2. 如果样本发生一点改动，会导致整棵树结构发生剧烈改变。

## 决策树算法对比

| 算法 | 支持模型 | 树结构 | 特征选择 | 连续值处理 | 缺失值处理 | 剪枝 |
|:----:|:----:|:----:|:----:|:----:|:----:|:----:|
| ID3 | 分类 | 多叉树 | 信息增益 | 不支持 | 不支持 | 不支持 |
| C4.5 | 分类 | 二叉树 | 信息增益比 | 支持 | 支持 | 支持 |
| CART | 分类，回归 | 二叉树 | 基尼系数，均方差 | 支持 | 支持 | 支持 |


## 决策树算法优缺点

**优点：**

1. 决策树简单直观，可解释性强

2. 基本不需要预处理、不需要归一化，不需要处理缺失值

3. 预测代价$ O(\log_2 m) $, $m$为样本数

4. 可同时处理连续值和离散值，较某些只能处理单一属性特征好

5. 可处理多分类问题

6. 可用交叉验证进行剪枝，泛化能力强

7. 对于异常点的容错能力好，健壮性高

**缺点：**

1. 决策树算法非常容易过拟合，导致泛化能力不强。可以通过设置节点最少样本数量和限制决策树深度来改进。

2. 决策树会因为样本发生一点点的改动，就会导致树结构的剧烈改变。这个可以通过集成学习之类的方法解决。

3. 寻找最优的决策树是一个NP难的问题，我们一般是通过启发式方法，容易陷入局部最优。可以通过集成学习之类的方法来改善。

4. 有些比较复杂的关系，决策树很难学习，比如异或。这个就没有办法了，一般这种关系可以换神经网络分类方法来解决。

5. 如果某些特征的样本比例过大，生成决策树容易偏向于这些特征。这个可以通过调节样本权重来改善。